In [1]:
# Data manipulation
import pandas as pd
import numpy as np

# Graph Neural Networks
import torch
import torch.nn.functional as F
from torch_geometric.data import Data as GeometricData
from torch_geometric.nn import GCNConv

# Bayesian Modeling
import pymc as pm

# Visualization
import matplotlib.pyplot as plt
import networkx as nx

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')


WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


## 2. Loading and preprocessing

In [8]:
df = pd.read_csv('../data/raw/train_data.csv')

df.replace(-1.0, np.nan, inplace=True)
df.replace('-1', np.nan, inplace=True)

# Convert date columns to datetime
df['launch_date'] = pd.to_datetime(df['launch_date'])
df['date'] = pd.to_datetime(df['date'])
df['ind_launch_date'] = pd.to_datetime(df['ind_launch_date'], errors='coerce')

df.to_csv('../data/intermig/train_data-withNaNs.csv', index=False)
df.head()

,brand,che_pc_usd,che_perc_gdp,cluster_nl,corporation,country,launch_date,date,drug_id,ind_launch_date,indication,insurance_perc_che,population,prev_perc,price_month,price_unit,public_perc_che,therapeutic_area,target
0,BRAND_354E,1.209114,1.665879,BRAND_354E_COUNTRY_88A3,CORP_D524,COUNTRY_88A3,2014-06-01,2014-06-01,DRUG_ID_8795,NaT,['IND_C3B6'],1.893333,1.008039,0.028367,1.006444,1.013784,1.835821,THER_AREA_980E,1.000784
1,BRAND_626D,NaN,NaN,BRAND_626D_COUNTRY_8B47,CORP_01C7,COUNTRY_8B47,2014-06-01,2014-06-01,DRUG_ID_E66E,2014-09-01,"['IND_1590', 'IND_ECAC']",NaN,1.023562,0.000047,NaN,1.626677,NaN,THER_AREA_96D7,1.000000
2,BRAND_45D9,1.209114,1.665879,BRAND_45D9_COUNTRY_88A3,CORP_39F7,COUNTRY_88A3,2014-06-01,2014-06-01,DRUG_ID_F272,NaT,['IND_B2EF'],1.893333,1.008039,0.001502,NaN,3.144874,1.835821,THER_AREA_96D7,1.002258
3,BRAND_D724,1.851280,2.051770,BRAND_D724_COUNTRY_445D,CORP_711A,COUNTRY_445D,2014-06-01,2014-06-01,DRUG_ID_1D4E,NaT,['IND_BAFB'],1.000000,1.253186,0.001304,NaN,1.213446,1.805970,THER_AREA_6CEE,1.068761
4,BRAND_4887,1.791199,2.059130,BRAND_4887_COUNTRY_D8B0,CORP_443D,COUNTRY_D8B0,2014-06-01,2014-06-01,DRUG_ID_AA88,NaT,['IND_3F31'],2.013333,1.639352,0.054467,1.018589,1.008708,1.880597,THER_AREA_6CEE,1.036312


In [9]:
# Fill missing target values with zeros (since target is NaN in the sample)
df['target'] = df['target'].fillna(0)
# Calculate the difference in months
df['months_since_launch'] = (
    (df['date'].dt.year - df['launch_date'].dt.year) * 12 +
    (df['date'].dt.month - df['launch_date'].dt.month)
)

# If you want to account for the day of the month
df['months_since_launch'] -= df['date'].dt.day < df['launch_date'].dt.day

# Convert to integer type
df['months_since_launch'] = df['months_since_launch'].astype(int)

# Display the updated DataFrame
df.head()

,brand,che_pc_usd,che_perc_gdp,cluster_nl,corporation,country,launch_date,date,drug_id,ind_launch_date,indication,insurance_perc_che,population,prev_perc,price_month,price_unit,public_perc_che,therapeutic_area,target,months_since_launch
0,BRAND_354E,1.209114,1.665879,BRAND_354E_COUNTRY_88A3,CORP_D524,COUNTRY_88A3,2014-06-01,2014-06-01,DRUG_ID_8795,NaT,['IND_C3B6'],1.893333,1.008039,0.028367,1.006444,1.013784,1.835821,THER_AREA_980E,1.000784,0
1,BRAND_626D,NaN,NaN,BRAND_626D_COUNTRY_8B47,CORP_01C7,COUNTRY_8B47,2014-06-01,2014-06-01,DRUG_ID_E66E,2014-09-01,"['IND_1590', 'IND_ECAC']",NaN,1.023562,0.000047,NaN,1.626677,NaN,THER_AREA_96D7,1.000000,0
2,BRAND_45D9,1.209114,1.665879,BRAND_45D9_COUNTRY_88A3,CORP_39F7,COUNTRY_88A3,2014-06-01,2014-06-01,DRUG_ID_F272,NaT,['IND_B2EF'],1.893333,1.008039,0.001502,NaN,3.144874,1.835821,THER_AREA_96D7,1.002258,0
3,BRAND_D724,1.851280,2.051770,BRAND_D724_COUNTRY_445D,CORP_711A,COUNTRY_445D,2014-06-01,2014-06-01,DRUG_ID_1D4E,NaT,['IND_BAFB'],1.000000,1.253186,0.001304,NaN,1.213446,1.805970,THER_AREA_6CEE,1.068761,0
4,BRAND_4887,1.791199,2.059130,BRAND_4887_COUNTRY_D8B0,CORP_443D,COUNTRY_D8B0,2014-06-01,2014-06-01,DRUG_ID_AA88,NaT,['IND_3F31'],2.013333,1.639352,0.054467,1.018589,1.008708,1.880597,THER_AREA_6CEE,1.036312,0


In [10]:

# Calculate Total Addressable Market (TAM)
df['tam'] = df['population'] * df['prev_perc']

# Handle missing values in TAM
df['tam'].fillna(df['tam'].median(), inplace=True)

# Encode categorical variables using Label Encoding for simplicity
from sklearn.preprocessing import LabelEncoder

categorical_cols = ['brand', 'corporation', 'country', 'drug_id', 'therapeutic_area', 'indication']
label_encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

df.head()

,brand,che_pc_usd,che_perc_gdp,cluster_nl,corporation,country,launch_date,date,drug_id,ind_launch_date,...,insurance_perc_che,population,prev_perc,price_month,price_unit,public_perc_che,therapeutic_area,target,months_since_launch,tam
0,113,1.209114,1.665879,BRAND_354E_COUNTRY_88A3,116,28,2014-06-01,2014-06-01,121,NaT,...,1.893333,1.008039,0.028367,1.006444,1.013784,1.835821,10,1.000784,0,0.028595
1,223,NaN,NaN,BRAND_626D_COUNTRY_8B47,0,30,2014-06-01,2014-06-01,223,2014-09-01,...,NaN,1.023562,0.000047,NaN,1.626677,NaN,9,1.000000,0,0.000048
2,155,1.209114,1.665879,BRAND_45D9_COUNTRY_88A3,28,28,2014-06-01,2014-06-01,236,NaT,...,1.893333,1.008039,0.001502,NaN,3.144874,1.835821,9,1.002258,0,0.001514
3,489,1.851280,2.051770,BRAND_D724_COUNTRY_445D,55,13,2014-06-01,2014-06-01,25,NaT,...,1.000000,1.253186,0.001304,NaN,1.213446,1.805970,7,1.068761,0,0.001634
4,161,1.791199,2.059130,BRAND_4887_COUNTRY_D8B0,34,43,2014-06-01,2014-06-01,149,NaT,...,2.013333,1.639352,0.054467,1.018589,1.008708,1.880597,7,1.036312,0,0.089291


In [ ]:
# Create a list of unique drugs
unique_drugs = df['drug_id'].unique()

# For now, drop the difficult indication column
node_features = df.drop(columns=['ind_launch_date',  ])

print(node_features.dtypes)
node_features.head()


brand                           int32
che_pc_usd                    float64
che_perc_gdp                  float64
cluster_nl                     object
corporation                     int32
country                         int32
launch_date            datetime64[ns]
date                   datetime64[ns]
drug_id                         int32
indication                      int32
insurance_perc_che            float64
population                    float64
prev_perc                     float64
price_month                   float64
price_unit                    float64
public_perc_che               float64
therapeutic_area                int32
target                        float64
months_since_launch             int32
tam                           float64
dtype: object


,brand,che_pc_usd,che_perc_gdp,cluster_nl,corporation,country,launch_date,date,drug_id,indication,insurance_perc_che,population,prev_perc,price_month,price_unit,public_perc_che,therapeutic_area,target,months_since_launch,tam
0,113,1.209114,1.665879,BRAND_354E_COUNTRY_88A3,116,28,2014-06-01,2014-06-01,121,193,1.893333,1.008039,0.028367,1.006444,1.013784,1.835821,10,1.000784,0,0.028595
1,223,NaN,NaN,BRAND_626D_COUNTRY_8B47,0,30,2014-06-01,2014-06-01,223,27,NaN,1.023562,0.000047,NaN,1.626677,NaN,9,1.000000,0,0.000048
2,155,1.209114,1.665879,BRAND_45D9_COUNTRY_88A3,28,28,2014-06-01,2014-06-01,236,183,1.893333,1.008039,0.001502,NaN,3.144874,1.835821,9,1.002258,0,0.001514
3,489,1.851280,2.051770,BRAND_D724_COUNTRY_445D,55,13,2014-06-01,2014-06-01,25,185,1.000000,1.253186,0.001304,NaN,1.213446,1.805970,7,1.068761,0,0.001634
4,161,1.791199,2.059130,BRAND_4887_COUNTRY_D8B0,34,43,2014-06-01,2014-06-01,149,41,2.013333,1.639352,0.054467,1.018589,1.008708,1.880597,7,1.036312,0,0.089291


In [6]:
# Create node features (we'll use the encoded features)
node_features = df.groupby('drug_id').mean().reset_index()

# Create edges based on shared therapeutic areas
edges = []
for i, drug_i in enumerate(unique_drugs):
    for j, drug_j in enumerate(unique_drugs):
        if i < j:
            # Check if drugs share the same therapeutic area
            ta_i = df[df['drug_id'] == drug_i]['therapeutic_area'].values[0]
            ta_j = df[df['drug_id'] == drug_j]['therapeutic_area'].values[0]
            if ta_i == ta_j:
                # Create a PyTorch Geometric Data object
                edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
                x = torch.tensor(node_features.drop(columns=['drug_id']).values, dtype=torch.float)

                data = GeometricData(x=x, edge_index=edge_index)


TypeError: agg function failed [how->mean,dtype->object]